# Multi-Compartment Spherical Mean Technique

The recently proposed Multi-Compartment Spherical Mean Technique (MC-SMT) model *(Kaden et al. 2016)* is a spherical convolution-based technique, which separates intra- from extra-axonal diffusion inside the spherical convolution kernel as

$$
\begin{align}
\begin{aligned}
 E_{\textrm{MC-SMT}}= \overbrace{\operatorname{FOD}}^{\textrm{Fiber Distribution}}\,*_{\mathbb{S}^2}\,\left[\underbrace{f_r\overbrace{E_r(\lambda_\parallel)}^{\textrm{Stick}}}_{\textrm{Intra-Axonal}}+\underbrace{(1-f_r)\overbrace{E_h(\lambda_\parallel|\lambda_\perp^{\textrm{tort}})}^{\textrm{Zeppelin}}}_{\textrm{Hindered Extra-Axonal}}\right].
\end{aligned}
\end{align}$$

The formulation of MC-SMT finds similarities in models such as NODDI, but stands apart in only considering the spherical mean of the signal at every acquisition shell, as used in the Spherical Mean Technique (SMT) *(Kaden et al. 2015)*.

SMT observes that if the FOD is a probability density (i.e. integrated to unity) then spherical mean of the signal and the convolution kernel must be the same
\begin{equation}
\int_{\mathbb{S}^2}E_b(\textbf{g})d\textbf{g}=\int_{\mathbb{S}^2}(\operatorname{FOD}\,*_{\mathbb{S}^2}\,K)_b(\textbf{g})d\textbf{g}=\int_{\mathbb{S}^2}K_b(\textbf{g})d\textbf{g}=\epsilon_K(b,\lambda_\perp,\lambda_\parallel).
\end{equation}
The estimation of the multi-compartment kernel using SMT enables the characterization of per-axon micro-environments, as the effects of axon dispersion and crossings are only contained in the FOD.

Advantages:
- Insensitive to axon dispersion.

Limitations:
- Biased in the presence of non-white matter tissues (CSF)
- Only accounts for average of tissue properties in crossing configurations, which potentially each have different properties.

# Modelfitting on Monte-Carlo-simulated data

In [1]:
from dmipy.signal_models import spherical_mean_models
stick_mean = spherical_mean_models.C1StickSphericalMean()
zeppelin_mean = spherical_mean_models.G2ZeppelinSphericalMean()

In [2]:
from dmipy.core import modeling_framework
mcsmt_mod = modeling_framework.MultiCompartmentModel(
    models=[stick_mean, zeppelin_mean])
mcsmt_mod.parameter_names

['G2ZeppelinSphericalMean_1_lambda_par',
 'G2ZeppelinSphericalMean_1_lambda_perp',
 'C1StickSphericalMean_1_lambda_par',
 'partial_volume_0',
 'partial_volume_1']

In [3]:
mcsmt_mod.set_tortuous_parameter('G2ZeppelinSphericalMean_1_lambda_perp',
    'C1StickSphericalMean_1_lambda_par','partial_volume_0', 'partial_volume_1')
mcsmt_mod.set_equal_parameter('G2ZeppelinSphericalMean_1_lambda_par', 'C1StickSphericalMean_1_lambda_par')

## Real data example

In [4]:
from dmipy.data import saved_data
scheme_hcp, data_hcp = saved_data.wu_minn_hcp_coronal_slice()

This data slice originates from Subject 100307 of the Human Connectome Project, WU-Minn Consortium (Principal Investigators: David Van Essen and Kamil Ugurbil; 1U54MH091657) funded by the 16 NIH Institutes and Centers that support the NIH Blueprint for Neuroscience Research; and by the McDonnell Center for Systems Neuroscience at Washington University.


In [5]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.imshow(data_hcp[:, 0, :, 0].T, origin=True)
plt.axis('off')
plt.title('HCP coronal slice B0');

In [6]:
mcsmt_fit_hcp = mcsmt_mod.fit(scheme_hcp, data_hcp, mask=data_hcp[..., 0]>0)

Using parallel processing with 8 workers.


/user/rfick/home/anaconda2/lib/python2.7/site-packages/dmipy-0.1.dev0-py2.7.egg/dmipy/signal_models/spherical_mean_models.py:137: RuntimeWarning: invalid value encountered in true_divide


Setup brute2fine optimizer in 0.0055558681488 seconds
Fitting of 8181 voxels complete in 34.431746006 seconds.
Average of 0.00420874538638 seconds per voxel.


In [7]:
fitted_parameters = mcsmt_fit_hcp.fitted_parameters

fig, axs = plt.subplots(1, len(fitted_parameters), figsize=[15, 15])
axs = axs.ravel()

for i, (name, values) in enumerate(fitted_parameters.items()):
    cf = axs[i].imshow(values.squeeze().T, origin=True)
    axs[i].set_title(name)
    axs[i].set_axis_off()
    fig.colorbar(cf, ax=axs[i], shrink=0.2)

It is also possible to calculate the Mean Squared Error (MSE) and the $R^2$ coefficient of determination.  
In MSE, the lower the better, while $R^2$ ranges between 0 and 1, with 1 being a perfect model fit.

In [8]:
mse = mcsmt_fit_hcp.mean_squared_error(data_hcp)
R2 = mcsmt_fit_hcp.R2_coefficient_of_determination(data_hcp)

fig, axs = plt.subplots(1, 2, figsize=[15, 15])
cf = axs[0].imshow(mse.squeeze().T, origin=True, vmax=1e-3, cmap='Greys_r')
fig.colorbar(cf, ax=axs[0], shrink=0.33)
axs[0].set_title('Mean Squared Error', fontsize=20)
axs[0].set_axis_off()
cf = axs[1].imshow(R2.squeeze().T, origin=True, vmin=.98, cmap='Greys_r')
fig.colorbar(cf, ax=axs[1], shrink=0.33)
axs[1].set_title('R2 - Coefficient of Determination', fontsize=20)
axs[1].set_axis_off();

/user/rfick/home/anaconda2/lib/python2.7/site-packages/dmipy-0.1.dev0-py2.7.egg/dmipy/core/fitted_modeling_framework.py:197: RuntimeWarning: invalid value encountered in divide


The MSE shows that the fitting error is very low overall, with only slightly higher errors in the CSF and much larger errors in the skull. The $R^2$ agree with the MSE results, having values very close to 1 overall, with lower values in the CSF and skull.

## References
- Kaden, Enrico, et al. "Multi-compartment microscopic diffusion imaging." NeuroImage 139 (2016): 346-359.
- Kaden, Enrico, Frithjof Kruggel, and Daniel C. Alexander. "Quantitative mapping of the per‚Äêaxon diffusion coefficients in brain white matter." Magnetic resonance in medicine 75.4 (2016): 1752-1763.